Model training code for [From Confidently Incorrect Models to Humble Ensembles](https://pair.withgoogle.com/explorables/uncertainty-ood/)

## Vanilla Keras MNIST Model

https://keras.io/examples/vision/mnist_convnet/

In [1]:
%%capture
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

!pip install tensorflowjs==3.21.0
import tensorflowjs as tfjs

In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 [==============================] - 1s 0us/step


In [4]:
model = keras.Sequential(
  [
    keras.Input(shape=input_shape),
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation="softmax"),
  ]
)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=10, validation_split=0.9)

Epoch 1/10
47/47 [==============================] - 3s 41ms/step - loss: 1.4266 - accuracy: 0.5923 - val_loss: 0.4866 - val_accuracy: 0.8594
Epoch 2/10
47/47 [==============================] - 2s 39ms/step - loss: 0.4260 - accuracy: 0.8705 - val_loss: 0.2798 - val_accuracy: 0.9217
Epoch 3/10
47/47 [==============================] - 2s 37ms/step - loss: 0.2778 - accuracy: 0.9185 - val_loss: 0.2117 - val_accuracy: 0.9375
Epoch 4/10
47/47 [==============================] - 3s 62ms/step - loss: 0.2078 - accuracy: 0.9393 - val_loss: 0.1721 - val_accuracy: 0.9491
Epoch 5/10
47/47 [==============================] - 1s 28ms/step - loss: 0.1783 - accuracy: 0.9478 - val_loss: 0.1548 - val_accuracy: 0.9543
Epoch 6/10
47/47 [==============================] - 3s 61ms/step - loss: 0.1553 - accuracy: 0.9532 - val_loss: 0.1374 - val_accuracy: 0.9585
Epoch 7/10
47/47 [==============================] - 1s 29ms/step - loss: 0.1295 - accuracy: 0.9598 - val_loss: 0.1256 - val_accuracy: 0.9617
Epoch 8/10
47

## Train overfit modes

- Reduces training data
- No dropout or regularization
- Train for a long time

In [5]:
def trainOverfitModel(i):
  model = keras.Sequential(
    [
      keras.Input(shape=input_shape),
      layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
      layers.MaxPooling2D(pool_size=(2, 2)),
      layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
      layers.MaxPooling2D(pool_size=(2, 2)),
      layers.Flatten(),
      layers.Dense(20, activation='relu'),
      layers.Dense(10, activation='softmax')
    ]
  )
  model.summary()
  model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

  n = 720
  xp_train = x_train[n*i:n*(i + 1)]
  yp_train = y_train[n*i:n*(i + 1)]

  model.fit(xp_train, yp_train, batch_size=720, epochs=2000, validation_split=0.2,)

  return model

In [ ]:
!mkdir ./tfjs_models

for i in range(0, 10):
  print('model:', i)

  model = trainOverfitModel(i)
  tfjs.converters.save_keras_model(model, './tfjs_models/overfit-epoch_2000/' + str(i) + '/')

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 48ms/step - loss: 1.3799e-05 - accuracy: 1.0000 - val_loss: 0.7138 - val_accuracy: 0.9097
Epoch 1515/2000
1/1 [==============================] - 0s 63ms/step - loss: 1.3780e-05 - accuracy: 1.0000 - val_loss: 0.7139 - val_accuracy: 0.9097
Epoch 1516/2000
1/1 [==============================] - 0s 64ms/step - loss: 1.3759e-05 - accuracy: 1.0000 - val_loss: 0.7140 - val_accuracy: 0.9097
Epoch 1517/2000
1/1 [==============================] - 0s 49ms/step - loss: 1.3738e-05 - accuracy: 1.0000 - val_loss: 0.7140 - val_accuracy: 0.9097
Epoch 1518/2000
1/1 [==============================] - 0s 59ms/step - loss: 1.3717e-05 - accuracy: 1.0000 - val_loss: 0.7141 - val_accuracy: 0.9097
Epoch 1519/2000
1/1 [==============================] - 0s 51ms/step - loss: 1.3699e-05 - accuracy: 1.0000 - val_loss: 0.7142 - val_accuracy: 0.9097
Epoch 1520/2000
1/1 [==============================] - 0s 52ms/step - loss: 1

## Download models

In [ ]:
!zip -r tfjs_models.zip tfjs_models/

from google.colab import files
files.download('tfjs_models.zip')